<a href="https://colab.research.google.com/github/NgoMinhToan/colab_massive_data/blob/main/mid_term.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random

In [ ]:
h1 = lambda a: (3*a)%5
h2 = lambda a: (2*a)%5


In [145]:
stringA = 'Dịch Covid-19 làm tăng nhu cầu nhân lực ở nhiều lĩnh vực, trong đó lao động ngành IT với kinh nghiệm dày dặn có thể đạt mức lương trên 100 triệu đồng/tháng. Thị trường lao động đã trải qua những biến động mạnh vào năm 2020. Mới đây, Adecco Vietnam, đơn vị chuyên về giải pháp nhân sự phát hành Bảng hướng dẫn lương 2021. Tài liệu này cung cấp thống kê mức lương của hai thành phố lớn tại Việt Nam là Hà Nội và TP.HCM dựa theo kinh nghiệm làm việc của một số vị trí trong các ngành như nhân sự, luật, tài chính ngân hàng, công nghệ thông tin (CNTT), bán hàng và marketing, y tế, kỹ thuật và chế tạo. Các số liệu là mức lương gộp hàng tháng, chưa bao gồm các phúc lợi khác. Cụ thể, với xu hướng thúc đẩy quá trình chuyển đổi kỹ thuật số, nhu cầu tuyển dụng các vị trí liên quan đến CNTT như kỹ sư dữ liệu, kỹ sư cơ sở hạ tầng hay giám đốc công nghệ đang tăng lên đáng kể. Bên cạnh đó, kỹ sư phần mềm và kiến trúc sư phần mềm với kinh nghiệm dày dặn có thể đạt mức lương trên 100 triệu đồng/tháng. Trong lĩnh vực chăm sóc sức khỏe, các doanh nghiệp có xu hướng tập trung vào việc nâng cao năng lực cạnh tranh của lực lượng lao động hiện có. Một số phương pháp phổ biến nhằm nâng cao hiệu suất của nhân viên bao gồm đào tạo nội bộ, đánh giá hiệu quả công việc và bổ nhiệm lại nhân sự. Do đó, các chuyên gia về phát triển nhân tài và bán hàng được săn đón với mức lương hàng tháng tương đối cao, lên đến 80 triệu đồng. Trong ngành tài chính, các vị trí được tìm kiếm hàng đầu là giám đốc đầu tư cổ phần tư nhân, giám đốc đầu tư cổ phiếu niêm yết, giám đốc ngân hàng đầu tư và trưởng bộ phận nghiên cứu cổ phiếu cho các công ty FDI; giám đốc mua bán và sáp nhập, giám đốc điều hành và giám đốc tài chính cho các doanh nghiệp được tài trợ bởi các nhà quản lý tài sản từ Nhật Bản, Hàn Quốc, Mỹ, châu Âu, Hongkong và Singapore. Nhu cầu tuyển dụng các vị trí này có thể được xem là sự dịch chuyển lao động đáng kể trên thị trường tài chính trong các năm 2020 và 2021. Một số lĩnh vực khác cũng đang mở rộng việc săn đón nhân tài bao gồm sản xuất chế tạo, công nghệ tài chính (Fintech), logistics, nông nghiệp và bán lẻ. Theo Adecco, 2020 là một năm đầy thách thức đối với nền kinh tế Việt Nam. Sự bùng phát của dịch Covid-19 đã làm gián đoạn các hoạt động thương mại và sản xuất. Ngành du lịch và hàng không là một trong những ngành bị ảnh hưởng nặng nề nhất. Dòng vốn FDI vào Việt Nam năm ngoái giảm 25%. Việc đóng cửa các doanh nghiệp và sa thải hàng loạt khiến tỷ lệ thất nghiệp ở khu vực thành thị trong quý II/2020 đạt mức cao nhất trong vòng 10 năm qua. Tuy nhiên, nhờ những nỗ lực trong việc ngăn chặn dịch bệnh, Việt Nam được xem là điểm sáng trên bản đồ kinh tế toàn cầu. Nền kinh tế tiếp tục đạt được tốc độ tăng trưởng dương và thuộc hàng cao nhất thế giới. Lĩnh vực sản xuất và xuất khẩu đang phục hồi mạnh mẽ sau những tác động ban đầu của đại dịch. Các hiệp định thương mại như: EVFTA, UKVFTA, RCEP được ký kết giúp Việt Nam đa dạng hóa thị trường xuất khẩu và đối tác thương mại. Đồng thời, ngày càng nhiều công ty tái phân bổ chuỗi sản xuất sang Việt Nam trong bối cảnh các nhà máy tại các nước khác đang đóng cửa và cuộc chiến thương mại Mỹ - Trung vẫn tiếp diễn'
stringB = 'Sự phát triển của AI, Big Data và bảo mật đã mở ra cơ hội mới cho nhân lực về công nghệ thông tin. Lương cho các vị trí kỹ sư IT hiện cũng có thể đạt hơn 100 triệu đồng/tháng. Theo "Bảng hướng dẫn lương 2021" của công ty giải pháp nhân sự Adecco Việt Nam, do xu hướng thúc đẩy quá trình phát triển kỹ thuật số, nhu cầu tuyển dụng nhân sự công nghệ thông tin ở các vị trí kỹ sư dữ liệu, kỹ sư cơ sở hạ tầng và giám đốc công nghệ tăng lên đáng kể. Trong đó, kiến trúc sư phầm mềm/giải pháp, kỹ sư phần mềm, kỹ sư dữ liệu có thể nhận được mức lương cao nhất lần lượt 160 triệu, 120 triệu và 80 triệu đồng mỗi tháng. Giám đốc công nghệ và giám đốc công nghệ thông tin trên 5 năm kinh nghiệm có thể nhận mức lương 250-400 triệu đồng/tháng tại TP HCM, 120-250 triệu đồng/tháng ở Hà Nội. Luong ky su cong nghe tren 100 trieu dong anh 1 Sự phát triển của AI, Big Data và bảo mật đã mở ra cơ hội mới cho các kỹ sư công nghệ thông tin. Ảnh: Adecco Việt Nam. Trong nhóm ngành chăm sóc sức khỏe, các vị trí quản trị tài năng và nhân lực bán hàng hiệu quả rất được săn đón, với mức lương hàng tháng lên đến 80 triệu đồng. Trong ngành tài chính, các vị trí hàng đầu là: giám đốc đầu tư cổ phần tư nhân, giám đốc đầu tư cổ phiếu niêm yết, giám đốc ngân hàng đầu tư, trưởng bộ phận nghiên cứu cổ phiếu cho các công ty FDI, giám đốc mua bán và sáp nhập, giám đốc điều hành, giám đốc tài chính cho doanh nghiệp được tài trợ bởi các nhà quản lý tài sản từ Nhật Bản, Hàn Quốc, Mỹ, châu Âu, Hong Kong và Singapore. Nhu cầu tuyển dụng các vị trí trên đây có thể được xem là sự dịch chuyển lao động đáng kể trên thị trường tài chính trong các năm 2020 và 2021. Trong đó, vị trí giám đốc đầu tư cổ phần tư nhân từ 5 năm kinh nghiệm có thể nhận mức lương 180-250 triệu đồng mỗi tháng tại TP HCM, 150-215 triệu đồng ở Hà Nội. Một số lĩnh vực khác cũng đang mở rộng việc săn đón nhân tài bao gồm: sản xuất chế tạo, công nghệ tài chính (Fintech), quản lý chuỗi cung ứng, nông nghiệp và bán lẻ. Luong ky su cong nghe tren 100 trieu dong anh 2 Giám đốc đầu tư cổ phần tư nhân từ 5 năm kinh nghiệm có thể nhận mức lương 180-250 triệu đồng mỗi tháng tại TP HCM, 150-215 triệu đồng ở Hà Nội. Ảnh: Adecco Việt Nam. Adecco Việt Nam cũng cho hay trong năm 2020, do ảnh hưởng của đại dịch Covid-19, tăng trưởng GDP năm 2020 của Việt Nam thấp nhất trong thập kỷ (2011 - 2020), ở mức 2,91%. Nhiều hoạt động thương mại và sản xuất bị gián đoạn, dòng vốn FDI vào Việt Nam giảm 25%. Việc đóng cửa doanh nghiệp và sa thải hàng loạt khiến tỷ lệ thất nghiệp ở khu vực thành thị đạt mức cao nhất trong 10 năm vào quý 2/2020. Du lịch và hàng không là một trong những ngành bị ảnh hưởng nặng nề nhất. Tuy nhiên, nhờ thành công trong việc ngăn chặn virus, Việt Nam được coi là điểm sáng trên bản đồ kinh tế toàn cầu. Đây còn là điểm đến của các nhà đầu tư nước ngoài bao gồm Mỹ, Singapore, Hàn Quốc, Trung Quốc, Nhật Bản...Việt Nam đứng thứ 33 trong danh sách 100 thương hiệu quốc gia giá trị nhất thế giới, tăng 9 bậc so với 2019'
stringC = 'Nhu cầu cao cùng nguồn cung khan hiếm đã khiến nghề kỹ sư IT cùng một số ngành nghề khác đang được săn đón với mức lương khiến nhiều người ngưỡng mộ. Theo VnExpress, công ty giải pháp nhân sự Adecco Việt Nam mới đây đã phát hành Bảng hướng dẫn lương 2021. Trong đó cho biết, nhu cầu tuyển dụng của một số vị trí thuộc ngành công nghệ thông tin đang có xu hướng tăng lên đáng kể. Cụ thể, 3 vị trí trong ngành này được đánh giá hot nhất là giám đốc công nghệ, kỹ sư dữ liệu và kỹ sư cơ sở hạ tầng. IT hiện là một trong những ngành được trả lương khá cao ở Việt Nam. (Ảnh: Grow Up Work) IT hiện là một trong những ngành được trả lương khá cao ở Việt Nam. (Ảnh: Grow Up Work) Với trung bình 5 năm kinh nghiệm, một người ở vị trí giám đốc công nghệ và giám đốc công nghệ thông tin có thể nhận mức lương thấp nhất là 120 triệu đồng và cao nhất lên đến 400 triệu đồng tùy khu vực. Ngoài ra các vị trí như kiến trúc sư phần mềm/giải pháp, kỹ sư phần mềm, kỹ sư dữ liệu cũng có thể nhận các mức lương tương ứng lần lượt là 160 triệu đồng, 120 triệu đồng và 80 triệu đồng mỗi tháng. Mức lương thuộc một số vị trí ngành Công nghệ thông tin hiện nay. (Ảnh: Adecco Việt Nam) Mức lương thuộc một số vị trí ngành Công nghệ thông tin hiện nay. (Ảnh: Adecco Việt Nam) Nguyên nhân của mức lương khủng này theo Adecco đánh giá là bởi quá trình chuyển đổi kỹ thuật số đang phát triển mạnh. Do đó, hiện giờ đang là kỷ nguyên bùng nổ cho các chuyên gia IT có năng lực. Sự phất lên của trí tuệ nhân tạo (AI), dữ liệu lớn (Big Data) cũng đòi hỏi một lượng lớn nhân lực có chuyên môn từ nghiệp vụ đến kỹ năng giao tiếp, khả năng ngôn ngữ. Song các ứng viên đủ điều kiện lại không nhiều nên lẽ thường tình, mức lương đưa ra cũng phải tăng lên để giữ chân người tài. Nguồn nhân sự cho ngành IT vẫn luôn cung ít hơn cầu. (Ảnh: VnExpress) Nguồn nhân sự cho ngành IT vẫn luôn cung ít hơn cầu. (Ảnh: VnExpress) Tuy nhiên theo như chia sẻ đến từ chính người trong nghề, tất cả những đánh giá này cũng chỉ là bề nổi mà thôi. Anh T.T cho biết: “Kinh nghiệm 5 năm mà có mức lương như trên thì cũng khá hiếm, phải làm việc trong công ty hàng top hoặc thực lực phải rất giỏi. Nói chung, lương lên được mức hơn trăm triệu cũng thuộc hàng “siêu nhân” rồi. Anh em bình thường đa số với từng đó năm kinh nghiệm cũng chỉ loay hoay 1/3 mức này thôi. Có làm rồi mới biết cực.” Có ý kiến lại cho rằng, với mức lương lý tưởng thế này, vô hình chung đã tạo động lực cho rất nhiều bạn trẻ mục tiêu để theo đuổi. Nhất là trong tình cảnh ngành IT chưa có dấu hiệu đủ nhân lực. Chỉ cần có năng lực, đủ đam mê thì không ngại học xong không có đầu ra. Để được hưởng mức lương cao này thì cũng phải có năng lực xứng tầm mới được. (Ảnh: VnExpress) Để được hưởng mức lương cao này thì cũng phải có năng lực xứng tầm mới được. (Ảnh: VnExpress) Đối với những cô gái không đủ sức “trâu bò” để theo đuổi công việc đặc thù này thì có thể nghĩ đến chuyện lấy một anh chồng làm nghề IT . Nói cho vui vậy chứ các cặp đôi không phải lúc nào cũng đến với nhau vì kinh tế. Chưa kể con trai làm IT đôi khi lại không thỏa mãn được tâm hồn lãng mạn của các bạn gái đâu. Mỗi ngành nghề đều có cái giá của nó và như nghề IT cũng vậy. Để có được lương cao thì đồng nghĩa với việc phải đánh đổi nhiều vì hiện tại làm gì có việc nhẹ lương cao tồn tại nữa đâu.'

stringA = stringA.lower()
stringB = stringB.lower()
stringC = stringC.lower()

In [146]:

k = 8
shingling = set()
for i in range(0, len(stringA) -k):
    shingling.add(stringA[i:i+k])

for i in range(0, len(stringB) -k):
    shingling.add(stringB[i:i+k])

for i in range(0, len(stringC) -k):
    shingling.add(stringC[i:i+k])

length_shingle = len(shingling)

print('The Union K-gram:', shingling)
print()

The Union K-gram: {'a sẻ đến', 'i việc p', 'ng các v', 'g nhân s', 'c cũng đ', ' lực. sự', 'kể. tron', ' it chưa', ' hong ko', 'ưa kể co', ' với việ', 'iệu quả ', 'khác cũn', 'tin. ảnh', 'tuy nhiê', 'một lượn', ' xuất sa', 'hiếu niê', 'ực có ch', ' kiếm hà', ' chính (', ' kiện lạ', 'hiện là ', ' và chế ', 'nghiệm l', 'u vì kin', ' hàng đư', ' mộ. the', 'vui vậy ', 'nam là h', 'dịch. cá', 'u để the', 'ng. it h', 'n mềm và', 'n bao gồ', 'ệp ở khu', ' thường ', 'hân sự p', 'i. nói c', 'h người ', 'ùng phát', ' nghệ và', 'chỉ là b', ' 25%. vi', ' rồi. an', '. it hiệ', 'ược trả ', 'p việt n', 'ng mại n', 'g ty giả', ', 3 vị t', ' nó và n', 'rường xu', 'ng danh ', 'ải làm v', 'a bao gồ', ', ở mức ', 't hơn 10', 'nội. luo', ' tốc độ ', 'y có thể', 'lương ch', 'các mức ', ' (ai), d', 'chính, c', 'án đoạn,', ' nhật bả', 'hàng top', '%. nhiều', 'con trai', 'hằm nâng', ' nhiên, ', 'vị trí đ', ' việc đó', 'ạt mức c', 'ngân hàn', 'ghệ tài ', 'ơn 100 t', ' lực phả', 'tế việt ', 'ý chuỗi ', 'nhân sự,

In [147]:

S1 = []
S2 = []
S3 = []
S = [S1, S2, S3]

for i in shingling:
    if i in stringA:
        S1.append(1)
    else:
        S1.append(0)

    if i in stringB:
        S2.append(1)
    else:
        S2.append(0)

    if i in stringC:
        S3.append(1)
    else:
        S3.append(0)

print('The matrix boolean: ')
print('S1:', S1)
print('S2:', S2)
print('S3:', S3)
print()


The matrix boolean: 
S1: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,

In [148]:
def Jaccard_similarity(a, b):
    total = 0
    index = 0
    for i in range(len(a)):
        if a[i] == 1 or b[i] == 1:
            total += 1
        if a[i] == b[i] == 1:
            index += 1
    return index/total

print('Jaccard similarity: ')      
print('S1 - S2:', Jaccard_similarity(S1, S2))
print('S1 - S3:', Jaccard_similarity(S1, S3))
print('S2 - S3:', Jaccard_similarity(S2, S3))
print()

Jaccard similarity: 
S1 - S2: 0.2774094849566548
S1 - S3: 0.07293322734499205
S2 - S3: 0.08372093023255814



In [149]:

random_permutation_1 = random.sample(range(length_shingle), length_shingle)
random_permutation_2 = random.sample(range(length_shingle), length_shingle)
random_permutation_3 = random.sample(range(length_shingle), length_shingle)
random_permutation = [random_permutation_1, random_permutation_2,random_permutation_3]

Signature_matrix = []
for perm in range(len(random_permutation)):
    sig_row = [length_shingle]*3
    for i in range(length_shingle):
        for s in range(len(S)):
            if S[s][i] == 1:
                sig_row[s] = min(random_permutation[perm][i], sig_row[s])
    Signature_matrix.append(sig_row)

print('The Signature matrix: ')
for i in Signature_matrix:
    print(i)
print()

The Signature matrix: 
[4, 4, 0]
[3, 3, 0]
[0, 0, 2]



In [150]:

def Probility_Sig(a, b):
    count = 0
    for i in range(len(a)):
        if a[i] == b[i]:
            count += 1
    return count/len(a)

print('Probility Sig: ')
print('S1 - S2:', Probility_Sig(list(map(lambda a: a[0], Signature_matrix)), list(map(lambda a: a[1], Signature_matrix))))
print('S1 - S3:', Probility_Sig(list(map(lambda a: a[0], Signature_matrix)), list(map(lambda a: a[2], Signature_matrix))))
print('S2 - S3:', Probility_Sig(list(map(lambda a: a[1], Signature_matrix)), list(map(lambda a: a[2], Signature_matrix))))
print()


Probility Sig: 
S1 - S2: 1.0
S1 - S3: 0.0
S2 - S3: 0.0



In [151]:
# create hash function
def hash_function(x, a, b, c):
    return (a*x + b) % c

hash_func_arr = []
num_for_hash = 10
# tim so nguyen to nho nhat nhung lon hon x
c = 0
for i in range(length_shingle, length_shingle + 100):
    prime = True
    for j in range(2, i//2):
        if i%j==0:
            prime = False
            break
    if prime:
        c = i
        break

for i in range(num_for_hash):
    hash_func_arr.append([random.randint(1, length_shingle), random.randint(1, length_shingle), c])
print(hash_func_arr)


[[5818, 4221, 6067], [305, 2881, 6067], [1266, 994, 6067], [624, 507, 6067], [1275, 1417, 6067], [3750, 2398, 6067], [5716, 3994, 6067], [2786, 2370, 6067], [3619, 1933, 6067], [276, 104, 6067]]


In [153]:

min_hash_table = [[1000000000 for i in range(num_for_hash)] for j in range(len(S))]
for r in range(length_shingle):
    hash_row = [hash_function(r, *a) for a in hash_func_arr]
    for c in range(len(S)):
        if S[c][r] == 1:
            for i in range(num_for_hash):
                if min_hash_table[c][i] > hash_row[i]:
                    min_hash_table[c][i] = hash_row[i]

for i in range(num_for_hash):
    print(min_hash_table[0][i], min_hash_table[1][i], min_hash_table[2][i])

0 0 1
2 0 1
0 4 1
0 0 4
2 0 3
3 0 1
4 1 0
1 0 2
0 2 9
1 2 0
